In [1]:
!pip install tqdm --quiet

In [2]:
import requests
from tqdm.notebook import tqdm
import os.path
from calendar import monthrange
import time

API_KEY = "FpIG19BfekWUFWGPhb5Nl_zIMPA7Z7gI7ZQ1DESEGkk"
# API_KEY = "zw_mUBWe3F72OcpgY1j2FjnVQMULgW52uOJzvvdxT6A"

# Note the months to fetch and generate dates
months = [10, 11, 12]
dates = [f"""2021-{month:02}-{(day+1):02}""" for month in months for day in range(monthrange(2022, month)[1])]

# Show last 10 days to fetch
print(len(dates))
dates[-10:]

92


['2021-12-22',
 '2021-12-23',
 '2021-12-24',
 '2021-12-25',
 '2021-12-26',
 '2021-12-27',
 '2021-12-28',
 '2021-12-29',
 '2021-12-30',
 '2021-12-31']

In [5]:
# Downloading real-time historical data.
!mkdir -p ./data/0-koda-gtfs-rt/vp/
!mkdir -p ./data/0-koda-gtfs-rt/tu/
# !open ./data/0-koda-gtfs-rt/tu/

headers = {'accept': 'application/octet-stream'}
iterator = tqdm(dates)
for date in iterator:
    iterator.set_description("Processing %s" % date)

    # Create parameter dictionary including key for this iteration
    params = {'date': date, 'key': API_KEY}

    # Construct filename and output path.
    # If files are already downloaded, we skip this iteration.
    file_path_vp = f"""./data/0-koda-gtfs-rt/vp/{params['date']}.7z"""
    file_path_tu = f"""./data/0-koda-gtfs-rt/tu/{params['date']}.7z"""

    # Trip Updates
    if os.path.exists(file_path_tu):
        print(date+" TU Exists")
    else:
        received = False
        while received == False:

            response_tu = requests.get('https://api.koda.trafiklab.se/KoDa/api/v2/gtfs-rt/sl/TripUpdates', 
                            params=params, headers=headers)
            print(type(response_tu.status_code), response_tu.status_code)

            if response_tu.status_code != 200:
                print(date + " TU: Sleeping and trying again")
                time.sleep(10)
                
            if response_tu.status_code == 200: 
                received = True
                print(date + "TU: Fetching ")
                with open(file_path_tu, 'wb') as file:
                    file.write(response_tu.content)


    # Vehicle Positions
    if os.path.exists(file_path_vp):
        print(date+" VP Exists")
    else:
        received = False
        while received == False:

            response_vp = requests.get('https://api.koda.trafiklab.se/KoDa/api/v2/gtfs-rt/sl/VehiclePositions', 
                                    params=params, headers=headers)
            print(type(response_vp.status_code), response_vp.status_code)

            if response_tu.status_code != 200:
                print(date + "VP: Sleeping and trying again")
                time.sleep(10)

            if response_vp.status_code == 200:
                received = True
                print(date + " VP: Fetching")
                with open(file_path_vp, 'wb') as file:
                    file.write(response_vp.content)

  0%|          | 0/92 [00:00<?, ?it/s]

2021-10-01 TU Exists
2021-10-01 VP Exists
2021-10-02 TU Exists
2021-10-02 VP Exists
2021-10-03 TU Exists
2021-10-03 VP Exists
2021-10-04 TU Exists
<class 'int'> 200


NameError: name 'response_tu' is not defined